In [1]:
import os
import cv2
import random
import shutil
import torchvision.transforms as transforms


from xeception_2x1ch import *
from xeception import *
from googlenetv4 import *
import pandas as pd
import numpy as np

d:\SHkim\contactlensCC_wxpython\wx_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
radius_list = []
def concat_origin_img(white_img, black_img):
    white_img = 255 - white_img

    img_sum = np.zeros((white_img.shape[0],white_img.shape[1],3), dtype=np.uint8)
    img_sum[:,:,0] = white_img
    img_sum[:,:,1] = black_img
    
    return img_sum

def find_circles(img_origin):
    global radius_list
    try:
        img = cv2.cvtColor(img_origin, cv2.COLOR_BGR2GRAY)
        #가우시안필터
        gaus = cv2.GaussianBlur(img, (5, 5), 7)

        #적응형이진화
        thres = cv2.adaptiveThreshold(gaus, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 5)
        thres = cv2.GaussianBlur(thres, (5, 5), 7)
        
        circles = cv2.HoughCircles(thres, cv2.HOUGH_GRADIENT, 1, 1200, param1 = 250, param2 = 60, minRadius = 720, maxRadius = 800)
    except Exception as e:
        print(e)
        return False, img    
    try:
        if len(circles) > 0:
            for i in circles[0]:
                cx = i[0]
                cy = i[1]
                radius = i[2]
                radius_list.append(radius)
    except:
        cx = img_origin.shape[1]//2
        cy = img_origin.shape[0]//2
        print("cannot find circles")

    #이미지 ROI 추출
    margin = 40 # 여백
    s_radius = 740
    x_min = int(cx - s_radius - margin)
    y_min = int(cy - s_radius - margin)
    x_max = int(cx + s_radius + margin)
    y_max = int(cy + s_radius + margin)
    img_roi = img_origin[y_min:y_max,x_min:x_max]
    img_roi = cv2.resize(img_roi,(1500,1500))
    # show_subplot_img([img_roi], 15)
        
    return True, img_roi

In [3]:
def copy_to_savefolder(img_path_list, result_list, class_list, save_dir):
    for n,result in enumerate(result_list):
        for i,r in enumerate(result):
            if r:
                shutil.copy(img_path_list[n], save_dir + '/' + class_list[i]+ '/' + os.path.basename(img_path_list[n]))

class_list = ['good', 'b_edge', 'burr', 'borken', 'b_bubble', 'etc', 'no_lens']

In [4]:
import test
import numpy as np
import time

base_path = r'E:\test_temp\2nd_model_imagedata'
save_dir = r'E:\test_temp\test_result\ggn'

for clsss in class_list:
    os.makedirs(save_dir + '/' + clsss, exist_ok=True)

start_time = time.time()
file_list, out = test.classfication(img_path=base_path,batch_size=20,selected_model='googlenetv4', weight_path="E:/weights/classification_weight/train_13/60_weight.pt")
copy_to_savefolder(file_list, out, class_list=class_list, save_dir=save_dir)
results_array = np.array(out)

# count number of True values per column
column_counts = np.sum(results_array, axis=0)

# count number of True values in total
total_count = len(out)

print("Count per column:", column_counts)
print("Total count:", total_count)
print("total time : ", time.time() - start_time)
print("average time : ", (time.time() - start_time) / total_count)

E:\test_temp\2nd_model_imagedata is a directory.
Count per column: [127085   2833    271   1456   3478   2729   5354]
Total count: 143629
total time :  5816.442410707474
average time :  0.040496295390954985
